In [25]:
import numpy as np
import scipy as sp
import sys
from numpy.random import uniform

from scipy.stats import multivariate_normal

X = np.genfromtxt('data/s1cluster.txt')

In [27]:
uniform(0,2, 5)

array([1.37023551, 1.39084976, 1.54583783, 1.97525242, 1.32077845])

In [33]:
for i, col in enumerate(X.T):
    print(col)
    print(i)

[664159. 665845. 597173. ... 650661. 599647. 684091.]
0
[550946. 557965. 575538. ... 861267. 858702. 842566.]
1


In [79]:
    minmaxrange = np.zeros(X.shape[1]).reshape(1,-1)
    for col in X.T:
        ranger = np.array([np.min(col), np.max(col)])
        minmaxrange = np.concatenate((minmaxrange, [ranger]))
    minmaxrange

array([[     0.,      0.],
       [ 19835., 961951.],
       [ 51121., 970756.]])

In [68]:
minmaxrange[1::]

array([[19835.0, 961951.0],
       [51121.0, 970756.0]], dtype=object)

In [54]:
for component in minmaxranges:
    rand_feat_val = uniform(component[0], component[1])

In [84]:
# this function will initialize the centroids off of the data

def initialize_centroids(X, cluster_num):
    minmaxrange = np.zeros(X.shape[1]).reshape(1,-1)
    for col in X.T:
        ranger = np.array([np.min(col), np.max(col)])
        minmaxrange = np.concatenate((minmaxrange, [ranger]))
    minmaxrange = minmaxrange[1::] #list of min/max ranges for each feature
    
    
    #this generates a vector within the min/max bounds of each dimension in the dataset
    initcentroids = np.zeros(cluster_num).reshape(1,-1)
    for feat_min, feat_max in minmaxrange:
        featclusterlist = uniform( feat_min, feat_max , size = cluster_num) #random initialization of feature
        initcentroids = np.concatenate((initcentroids, [featclusterlist]))
    
    return initcentroids[1::].T


#outputs cluster assignment for each observation in dataset (index in centroidlist)
def clusterassign(X, centroidlist):
    
    #helper function to cluster assign for a given data point
    def vecclusterassign(x, centroidlist):
        #calculate squared distance of point to each centroid
        distlist = [ np.dot((x - centroid), (x - centroid)) for centroid in centroidlist ] 
        return np.argmin(distlist) # gets centroid with minimum distance
    
    clust_assignlist = np.array([vecclusterassign(x, centroidlist) for x in X])  # cluster assignment for each data point
    
    return np.array(clust_assignlist)

# takes 
def calc_clustermeans(X, assign_list):
    newcentroidlist = [np.average( X[np.where(assign_list == clust_identifier)[0]], axis = 0) for clust_identifier in np.unique(assign_list)]
    return newcentroidlist
    

In [85]:
initialize_centroids(X, 5)

array([[483288.91493387, 483053.46833862],
       [535223.45666312, 246765.12263975],
       [877258.26325105, 413632.81018074],
       [850728.57638637, 391345.5242613 ],
       [674449.45918955, 472304.33600624]])

In [3]:
h = initialize_centroids(X, 5)
m = clusterassign(X, h)

In [4]:
def KMeans(X, clust_num, iter_num):
    
    iterations = 0
    current_centroid = initialize_centroids(X, clust_num)
    
    
    # for assignment we do 10 iterations, but in practice will be a more general stopping condition
    while (iterations < iter_num):
        # assign cluster 
        current_assignment = clusterassign(X, current_centroid)
        
        # now update current centroid
        current_centroid = calc_clustermeans(X, current_assignment)
        centroid_outputter = np.array(current_centroid)
        
        filename = "centroids-" + str(iterations+1) + ".csv" #"i" would be each iteration
        np.savetxt(filename, centroid_outputter, delimiter=",")
        
        iterations +=1
    
    return centroid_outputter, current_assignment
        
    

In [5]:
KMeans(X, 15, 2)

(array([[540103.01652893, 579410.76033058],
        [168295.41492537, 347724.81791045],
        [244069.21238938, 847518.09144543],
        [320318.89085546, 161713.24483776],
        [859007.16571429, 552562.2       ],
        [825294.3974359 , 372653.30769231],
        [805371.30321593, 240956.04287902],
        [822447.65472313, 734224.9771987 ],
        [621610.03537736, 489045.01650943],
        [417332.14240506, 787518.76265823],
        [670929.06818182, 862765.73295455],
        [154248.22077922, 557568.4961039 ],
        [509216.3933518 , 179463.26869806],
        [380289.215625  , 469141.825     ]]),
 array([ 8,  8,  8, ..., 10, 10, 10], dtype=int64))

In [29]:
#initialize means
def initialize_means(X, cluster_num):
    minmaxranges = []
    for col in X.T:
        minmaxcolrange = [np.min(col), np.max(col)]
        minmaxranges.append(minmaxcolrange)
    
    #this generates a vector within the min/max bounds of each dimension in the dataset
    initcentroids = np.array([ np.array([random.uniform(componentrange[0], componentrange[1]) for componentrange in minmaxranges]) for i in range(cluster_num) ])
    
    return initcentroids
#--------------------------------------------------------------------------------------------------------------------------------------
#cluster_posterior is our q-distribution over which we take the expectation of the joint distribution on x and its cluster assignment

def cluster_posterior(x, pi, clust_means, clust_covars):
    
    clusternums = len(pi)
    norm = sum([ pi[j] * multivariate_normal.pdf(x, mean = clust_means[j], cov = clust_covars[j]) for j in range(clusternums)])
    
    if norm == 0.0:
        return (1/clusternums)*np.ones(clusternums)
    else:
        posterior = np.array([(1/norm) * clust_prob[k] * multivariate_normal.pdf(x, mean = clust_means[k], cov = clust_covars[k]) for k in clusternums])
        
    return posterior

#----------------------------------------------------------------------------------------------------------------------------------------------------
#given the current cluster probabilities, cluster means, cluster covariances: maximize q-expectation of joint distribution of X and cluster assignment
#output new point estimates of cluster probabilities, cluster means, cluster covariances

def Mstepupdate(X, clust_prob, clust_means, clust_covars):
    n = len(X)
    d = len(X.T)

    #initializes new cluster probability, cluster mean, and covariance lists
    Pi = []
    clust_mu = []
    clust_sigmas = []
    
    for k in np.arange(len(clust_prob)):
        # outputs Pi_k for kth cluster and calculates n_k
        nk = sum([cluster_posterior(x, k, clust_prob, clust_means, clust_covars) for x in X ])
        Pi.append(nk / n)
        
        #outputs new mean for kth cluster
        mu_k = (1 / nk)* sum( [ cluster_posterior(x, k, clust_prob, clust_means, clust_covars)*x for x in X ] )
        clust_mu.append(mu_k)
        
        #ouputs new covariance matrix for kth cluster
        Sigma_k = (1 / nk)* sum( [cluster_posterior(x, k, clust_prob, clust_means, clust_covars)*np.outer( (x - mu_k) , (x - mu_k) ) for x in X] )
        clust_sigmas.append(Sigma_k)
    
    return Pi, clust_mu, clust_sigmas



        
#EM_GMM(X, 15, 1)        

#### Define main GMM algorithm

In [26]:
def EM_GMM(X, clust_num, iter_num):
    
    d = np.shape(X)[1]
    n = np.shape(X)[0]
    
    # initialize Gaussian mixture distribution parameters -- python list format
    iterations = 0
    
    pi = (1/clust_num)*np.ones(clust_num) # initialize uniform cluster probability
    
    cluster_means = initialize_means(X, clust_num) #takes data and initializes cluster means based off of dataset limits
    """"
    cluster_sigmas = [np.identity(d) for k in np.arange(clust_num) ]
    
    while (iterations < iter_num ):
        #M-step updates (E-step hidden inside via inner call to cluster-posterior )
        cluster_proba, cluster_means, cluster_sigmas = Mstepupdate(X, cluster_proba, cluster_means, cluster_sigmas)
        
        # convert new Gaussian mixture parameters to correct output format
        pi_ouputter = np.array(cluster_proba)
        means_outputter = np.array(cluster_means)
        
            
        filename = "pi-" + str(iterations+1) + ".csv" 
        np.savetxt(filename, pi_outputter, delimiter=",") 
        
        filename = "mu-" + str(iterations+1) + ".csv"
        np.savetxt(filename, means_outputter, delimiter=",")
        
        for j in np.arange(len(cluster_sigmas)):
            filename = "Sigma-" + str(j+1) + "-" + str(iterations+1) + ".csv"
            np.savetxt(filename, cluster_sigmas[j], delimiter = ",")
        
        iterations += 1
        
        """
    print(cluster_means)

In [16]:

multivariate_normal.pdf([555000,130000], mean = [555000,130000], cov = np.identity(2))

0.15915494309189535

In [27]:
EM_GMM(X,15,1)

[[586736.3097070223   730629.789605211   ]
 [284220.27983774414  546509.5503852363  ]
 [158143.29017728544  808683.1632292408  ]
 [ 25514.712740782146 367402.5337826628  ]
 [539895.293584768    173951.6728083283  ]
 [624151.920715032    752796.4546158852  ]
 [766537.4941715605    71288.36336425233 ]
 [213535.72117586804  954569.8329605921  ]
 [365017.2843011949   843156.3242977345  ]
 [728352.2957706093   657195.486801179   ]
 [289243.5240510221   501359.74083790166 ]
 [474959.32811422215  488709.6590400039  ]
 [381413.1824704577   817942.533729387   ]
 [253635.27137001345  593475.2117256045  ]
 [593974.3625685638    87695.39049013663 ]]


In [28]:
pi

NameError: name 'pi' is not defined